# 🎯 Hull Tactical Market Prediction - Training & Inference

**Kaggle에서 모델 학습 후 실시간 예측 API 제공**

## 📌 Setup Instructions

### Step 1: Upload Files to Kaggle Dataset

1. **Go to**: https://www.kaggle.com/datasets
2. **Click**: "New Dataset"
3. **Upload files**:
   - All `.py` files from `src/`
   - All `.py` files from `scripts/`
   - `params.yaml` from `conf/`
4. **Name it**: `mydata`
5. **Click**: "Create"

### Step 2: Configure Dataset Name

셀 2에서 데이터셋 이름 확인:

```python
DATASET_NAME = "mydata"  # 여기 확인!
```

### Step 3: Submit to Competition

1. **Competition Setting**: Enable "Internet" OFF
2. **Accelerator**: None (CPU)
3. **Click**: "Submit to Competition"
4. **Wait**: 학습 완료 후 실시간 inference 시작

## 🚀 How It Works:

1. **Train return model** (~10 min)
2. **Train risk model** (~10 min)
3. **Optimize position strategy** (~5 min)
4. **Load trained models**
5. **Start inference server** for real-time predictions

**Total time: ~25 minutes + inference**

**Good luck! 🚀**

## 1️⃣ Setup: Configure Dataset Path

In [ ]:
import os
import sys
from pathlib import Path
import yaml

# ========== CONFIGURATION: 데이터셋 이름 (여기만 수정하세요!) ==========
DATASET_NAME = "my-hull-models"  # ← 업로드한 dataset 이름
# ====================================================================

print("="*80)
print("SETTING UP PATHS")
print("="*80)

# Kaggle 데이터셋 경로
DATASET_PATH = Path(f"/kaggle/input/{DATASET_NAME}")

if DATASET_PATH.exists():
    # Add dataset path to sys.path for imports
    sys.path.insert(0, str(DATASET_PATH))
    print(f"✓ Dataset found: {DATASET_PATH}")
    print(f"✓ Added to sys.path: {DATASET_PATH}")
    
else:
    print(f"❌ Dataset not found: {DATASET_PATH}")
    input_dir = Path("/kaggle/input/")
    if input_dir.exists():
        print("\n📁 Available datasets:")
        for item in input_dir.iterdir():
            print(f"  - {item.name}")
    raise FileNotFoundError(f"Dataset '{DATASET_NAME}' not found!")

# ========== CONFIG 복사 ==========
config_path = DATASET_PATH / "conf" / "params.yaml"
if config_path.exists():
    working_config_dir = Path("/kaggle/working/conf")
    working_config_dir.mkdir(parents=True, exist_ok=True)
    
    import shutil
    shutil.copy(config_path, working_config_dir / "params.yaml")
    
    # conf 디렉토리를 Python path에 추가
    sys.path.insert(0, str(working_config_dir.parent))
    print(f"\n✓ Config copied to: {working_config_dir}/params.yaml")
else:
    print(f"\n⚠️  Config file not found: {config_path}")

print("\n✅ Setup complete!")
print(f"Python path includes:")
for p in sys.path[:3]:
    print(f"  - {p}")

## 2️⃣ Train Return Model (~10 min)

In [ ]:
print("="*80)
print("STEP 1/4: TRAINING RETURN MODEL")
print("="*80)

# Return model 학습 스크립트 실행
script_path = DATASET_PATH / "scripts" / "optimize_return_model.py"

if script_path.exists():
    print(f"▶ Running: {script_path.name}\n")
    %run {script_path}
    print("\n✅ Return model training complete!")
else:
    raise FileNotFoundError(f"Script not found: {script_path}")

## 3️⃣ Train Risk Model (~10 min)

In [ ]:
print("="*80)
print("STEP 2/4: TRAINING RISK MODEL")
print("="*80)

# Risk model 학습 스크립트 실행
script_path = DATASET_PATH / "scripts" / "optimize_risk_model.py"

if script_path.exists():
    print(f"▶ Running: {script_path.name}\n")
    %run {script_path}
    print("\n✅ Risk model training complete!")
else:
    raise FileNotFoundError(f"Script not found: {script_path}")

## 4️⃣ Optimize Position Strategy (~5 min)

In [ ]:
print("="*80)
print("STEP 3/4: OPTIMIZING POSITION STRATEGY")
print("="*80)

# Position strategy 최적화 스크립트 실행
script_path = DATASET_PATH / "scripts" / "optimize_position_strategy.py"

if script_path.exists():
    print(f"▶ Running: {script_path.name}\n")
    %run {script_path}
    print("\n✅ Position strategy optimization complete!")
else:
    raise FileNotFoundError(f"Script not found: {script_path}")

## 5️⃣ Load Trained Models & Start Inference Server

In [ ]:
print("="*80)
print("STEP 4/4: LOADING TRAINED MODELS & STARTING INFERENCE SERVER")
print("="*80)

import pickle
import json
import polars as pl
import numpy as np
import pandas as pd
from pathlib import Path

# 학습된 모델 경로 (Kaggle working directory에 생성됨)
MODEL_DIR = Path("/kaggle/working/artifacts")

# ========== Return models 로드 ==========
return_models = []
return_feature_names = []

return_model_dir = MODEL_DIR / "models_optimized"
if return_model_dir.exists():
    # Models 로드
    for fold_idx in range(4):
        fold_path = return_model_dir / f"lightgbm_fold_{fold_idx}.pkl"
        if fold_path.exists():
            with open(fold_path, 'rb') as f:
                model = pickle.load(f)
                return_models.append(model)
            print(f"✓ Loaded return model fold {fold_idx}")
    
    # 모델에서 직접 feature names 가져오기 (가장 신뢰할 수 있는 방법)
    if return_models:
        return_feature_names = return_models[0].feature_name()
        print(f"✓ Extracted return feature names from model: {len(return_feature_names)} features")
    else:
        raise ValueError("No return models loaded!")
else:
    raise FileNotFoundError(f"Return models not found: {return_model_dir}")

# ========== Risk models 로드 ==========
risk_models = []
risk_feature_names = []

risk_model_dir = MODEL_DIR / "models_risk_optimized"
if risk_model_dir.exists():
    # Models 로드
    for fold_idx in range(4):
        fold_path = risk_model_dir / f"lightgbm_fold_{fold_idx}.pkl"
        if fold_path.exists():
            with open(fold_path, 'rb') as f:
                model = pickle.load(f)
                risk_models.append(model)
            print(f"✓ Loaded risk model fold {fold_idx}")
    
    # 모델에서 직접 feature names 가져오기 (가장 신뢰할 수 있는 방법)
    if risk_models:
        risk_feature_names = risk_models[0].feature_name()
        print(f"✓ Extracted risk feature names from model: {len(risk_feature_names)} features")
    else:
        raise ValueError("No risk models loaded!")
else:
    raise FileNotFoundError(f"Risk models not found: {risk_model_dir}")

# ========== Position strategy 로드 ==========
strategy_path = MODEL_DIR / "best_position_strategy.json"
mapper_k = 1.0  # Default
mapper_b = 2.0  # Default

if strategy_path.exists():
    with open(strategy_path, 'r') as f:
        strategy_params = json.load(f)
    
    # SharpeScalingMapper에 필요한 파라미터만 추출
    mapper_k = strategy_params.get('k', 1.0)
    mapper_b = strategy_params.get('b', 2.0)
    
    print(f"✓ Loaded position strategy: k={mapper_k}, b={mapper_b}")
else:
    print(f"⚠️  Position strategy not found, using defaults: k={mapper_k}, b={mapper_b}")

# ========== Import modules for inference ==========
from src.features import FeatureEngineering
from src.position import SharpeScalingMapper

print("\n" + "="*80)
print("DEFINING PREDICTION FUNCTION")
print("="*80)

def predict(test: pl.DataFrame) -> float:
    """
    Real-time prediction function called by Kaggle API.
    
    Args:
        test: Polars DataFrame with batch of test features
        
    Returns:
        allocation: float between 0.0 and 2.0
    """
    try:
        # Convert to pandas for feature engineering
        test_pd = test.to_pandas()
        
        # Feature engineering
        fe = FeatureEngineering()
        test_features = fe.transform(test_pd)

        # Use ONLY the features that were used during training
        if return_feature_names:
            missing_return = [c for c in return_feature_names if c not in test_features.columns]
            if missing_return:
                print(f"⚠️ Missing {len(missing_return)} return features (filled with 0)")
            
            # 정확히 학습 시 순서대로 피처 생성
            X_return = pd.DataFrame(index=test_features.index)
            for feat in return_feature_names:
                if feat in test_features.columns:
                    X_return[feat] = test_features[feat]
                else:
                    X_return[feat] = 0.0
        else:
            raise ValueError("Return feature names not loaded!")
        
        if risk_feature_names:
            missing_risk = [c for c in risk_feature_names if c not in test_features.columns]
            if missing_risk:
                print(f"⚠️ Missing {len(missing_risk)} risk features (filled with 0)")
            
            # 정확히 학습 시 순서대로 피처 생성
            X_risk = pd.DataFrame(index=test_features.index)
            for feat in risk_feature_names:
                if feat in test_features.columns:
                    X_risk[feat] = test_features[feat]
                else:
                    X_risk[feat] = 0.0
        else:
            raise ValueError("Risk feature names not loaded!")
        
        # Predict return (r_hat) - 앙상블
        r_hat_preds = [model.predict(X_return) for model in return_models]
        r_hat = float(np.mean([np.mean(pred) for pred in r_hat_preds]))
        
        # Predict risk (sigma_hat) - 앙상블
        sigma_hat_preds = [model.predict(X_risk) for model in risk_models]
        sigma_hat = float(np.mean([np.mean(pred) for pred in sigma_hat_preds]))
        
        # Map to position using strategy
        mapper = SharpeScalingMapper()
        allocation = mapper.map_positions(
            r_hat=np.array([r_hat]),
            sigma_hat=np.array([sigma_hat]),
            k=mapper_k,
            b=mapper_b
        )[0]
        
        # Ensure within bounds [0, 2]
        allocation = max(0.0, min(2.0, float(allocation)))
        
        return allocation
        
    except Exception as e:
        print(f"⚠️  Prediction error: {e}")
        import traceback
        traceback.print_exc()
        return 0.0  # Safe default

print("✓ Prediction function defined!")

# ========== Start inference server ==========
print("\n" + "="*80)
print("STARTING INFERENCE SERVER")
print("="*80)

import os
import kaggle_evaluation.default_inference_server

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # Production: Serve API for hidden test set
    print("🚀 Running in COMPETITION mode - serving real-time predictions")
    inference_server.serve()
else:
    # Local testing: Run on public test set
    print("🧪 Running in LOCAL mode - testing on public data")
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

print("\n✅ Inference server started!")